# SentenceBERT 한국어 문장 클러스터링 
- 배우 벤, 로버트, 드 니로, 클러스터8
비현실

클러스터 14 직장 내 노인

클러스터 15 할아버지에 대한 찬사 + 중립 10

클러스터 16  + 클러스터 18 + 중립 7

중립의 17 벤, 어른

노인, 연륜, 버지가 들어간 리뷰들만으로

노인일 때의 면집단


In [55]:
import pandas as pd
# all_review_sentiment_df = pd.read_csv("/Users/seongrok.kim/Github/TIL/NLP/NoCountryForOldMan/data/intern_all_review_sentiment.csv")
positive_review_sentiment_df = pd.read_csv("/Users/seongrok.kim/Github/TIL/NLP/NoCountryForOldMan/data/cluster_positive_df.csv")
neutral_review_sentiment_df = pd.read_csv("/Users/seongrok.kim/Github/TIL/NLP/NoCountryForOldMan/data/cluster_neutral_df.csv")

In [44]:
positive_review_sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44657 entries, 0 to 44656
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   44657 non-null  object
 1   cluster  44657 non-null  object
dtypes: object(2)
memory usage: 697.9+ KB


In [70]:
review_sentiment_df = positive_review_sentiment_df[positive_review_sentiment_df['cluster'] == f'Cluster_16']
review_sentiment_df_tmp_18 = positive_review_sentiment_df[positive_review_sentiment_df['cluster'] == f'Cluster_18']
review_sentiment_df_tmp = neutral_review_sentiment_df[neutral_review_sentiment_df['cluster'] == f'Cluster_17']
review_sentiment_df = pd.concat([review_sentiment_df, review_sentiment_df_tmp_18, review_sentiment_df_tmp])

In [66]:
review_sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8333 entries, 33713 to 8040
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   8333 non-null   object
 1   cluster  8333 non-null   object
dtypes: object(2)
memory usage: 195.3+ KB


In [ ]:

review_sentiment_df = positive_review_sentiment_df[positive_review_sentiment_df['cluster'] == f'Cluster_{cluster_num}']

In [72]:
from sentence_transformers import SentenceTransformer
cluster_num = 14
model_path = 'jhgan/ko-sroberta-multitask'
embedder = SentenceTransformer(model_path)
corpus = review_sentiment_df_tmp['review'].values.tolist()

corpus_embeddings_17 = embedder.encode(corpus) 

In [33]:
import numpy as np
# Load the array from the file
corpus_embeddings_all = np.load('./data/SentenceBERT_all.npy')
corpus_embeddings_all

(72286, 768)

In [21]:
len(all_review_sentiment_df)

72286

In [35]:
all_review_sentiment_df['embedding'] = list(corpus_embeddings_all)

In [70]:
all_review_sentiment_df.to_csv("/Users/seongrok.kim/Github/TIL/NLP/NoCountryForOldMan/data/intern_all_review_sentiment.csv", index=False, encoding='utf-8-sig')

In [67]:
def convert_cgv_score(tmp):
    if tmp == 1.0:
        re_score = 5.0
    elif tmp == 2.0:
        re_score = 5.0
    else:
        re_score = 1.0
    return re_score

In [4]:
def replace_score(df):
    df.loc[(df['platform'] == 'cgv') & (df['Review Score'].isin([0.0, 2.0])), 'Review Score'] = 5.0
    return df

all_review_sentiment_df = replace_score(all_review_sentiment_df)

In [36]:
positive_review_sentiment_df = all_review_sentiment_df.loc[(all_review_sentiment_df['analysis'] == 'positive') & (all_review_sentiment_df['Review Score'] >= 2.5)]

In [ ]:
np.array(positive_review_sentiment_df['embedding'])

In [37]:
import numpy as np
positive_corpus = np.array(positive_review_sentiment_df['embedding'].to_list())
positive_corpus.shape

(49932, 768)

In [49]:
corpus_embeddings_14.shape

(2945, 768)

In [73]:
from sklearn.cluster import KMeans
num_clusters = 10
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings_17)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
cluster_dict = {}
for i, cluster in enumerate(clustered_sentences):
    cluster_dict[f"Cluster_{i}"] = cluster

/Users/seongrok.kim/miniforge3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [61]:
for k, v in cluster_dict.items():
    print(k, len(v))

Cluster_0 81
Cluster_1 254
Cluster_2 148
Cluster_3 306
Cluster_4 99
Cluster_5 222
Cluster_6 53
Cluster_7 88
Cluster_8 108
Cluster_9 276


In [62]:
cluster_df = pd.DataFrame()

for k,v in cluster_dict.items():
    df = pd.DataFrame({"review": v})
    df['cluster'] = k
    cluster_df = pd.concat([cluster_df, df])

In [74]:
cluster_df = cluster_df.drop_duplicates(subset=['review'])
cluster_df.to_csv('./data/cluster_n17_df.csv', index=False, encoding='utf-8-sig')

In [64]:
cluster_df

,review,cluster
0,특히 연로하신 아버지와 같이 봐 더 감동적이었어요,Cluster_0
1,부모님도 좋으셨대요,Cluster_0
2,아빠가 보고 감동받으셨다고 함,Cluster_0
3,친구랑 봤는데남자친구랑도 보려고요할아버지 멋있어요 ㅎㅎ,Cluster_0
4,아버지께도 추천하고 싶다.,Cluster_0
...,...,...
271,고령화 시대 은퇴세대와 어떻게 손을 잡을 수 있는지 현명한 방법을 제시.,Cluster_9
272,그 60대 주인공은 감독 자신을 투영한 이상형의 남성인 것 같다.,Cluster_9
273,이외의 어르신들께서는 다 어디로 가서 어떻게 계실까,Cluster_9
274,고전적이고 무해한 할아버지 내니의 회사 돌보기.,Cluster_9
